In [6]:
import rospy
import numpy as np
#from turtlesim.srv import Spawn
from turtlesim.msg import Pose
#from geometry_msgs.msg import Twist
from pkg_tsr.srv import Task
from std_msgs.msg import Float32
from std_msgs.msg import String
import math

rospy.init_node("RobotSystem")

class Punkt():
    x = 0.0
    y = 0.0
    def __init__(self,x,y):
        self.x = x
        self.y = y

In [7]:
class RobotData():
    name = "NoName"
    battery = 100
    position = Pose()
    def __init__(self,name):
        self.name = name
        self.battery_sub = rospy.Subscriber(name+'/battery',Float32, self.callback_battery)
        self.pose_sub = rospy.Subscriber(name+'/pose',Pose, self.callback_pose)
        self.go_to_goal = rospy.ServiceProxy(name+'/go_to_goal',Task)
        self.req_pub = rospy.Publisher(name+'/polecenia',String, queue_size = 10)
        
    def callback_battery(self,data):
        self.battery = data.data
    def callback_pose(self,data):
        self.position = data
        
    def state(self):
        print(self.name + ':')
        print("battery: " + str(self.battery))
        print(self.position)
        
    def unregister(self):
        self.battery_sub.unregister()
        self.pose_sub.unregister()

In [8]:
class RobotSystem():
    robots = []
    def __init__(self,names):
        for name in names:
            self.robots.append(RobotData(name))
        self.req_sub = rospy.Subscriber('polecenia',String, self.callback_req)
        self.robot_info_sub = rospy.Subscriber('komunikaty_robotow',String,self.callback_info)
        self.user_info_pub = rospy.Publisher('komunikaty', String, queue_size = 10)
        
    def go_to(self,numerRobota,x,y,info):
        req_text = "idz," + str(x) + ','+ str(y) +',' + info
        self.robots[numerRobota - 1].req_pub.publish(req_text)
        
    def charge(self, numerRobota):
        self.user_info_pub.publish(f"System: Robot R{numerRobota} wysłany do ładowania")
        pos = self.robots[numerRobota - 1].position
        d1 = math.sqrt( (pos.x - 1)**2 + (pos.y - 1)**2 )
        d2 = math.sqrt( (pos.x - 10)**2 + (pos.y - 10)**2 )
        if d1 <= d2:
            self.go_to(numerRobota,1,1,"Dotarłem do stacji ładującej")
        else:
            self.go_to(numerRobota,10,10,"Dotarłem do stacji ładującej")
            
    def podlej(self, numerRobota, numerGrzadki):
        self.user_info_pub.publish(f"System: Robot R{numerRobota} wysłany do podlania grządki {numerGrzadki}")
        pos = self.robots[numerRobota - 1].position
        if pos.y >= 5.5:
            if numerGrzadki == 1:
                self.go_to(numerRobota,3,9,f"Rozpoczynam podlewanie grzadki {numerGrzadki}")
                self.go_to(numerRobota,3,2,f"Zakończyłem podlewanie grzadki {numerGrzadki}")
            elif numerGrzadki == 2:
                self.go_to(numerRobota,5.5,9,f"Rozpoczynam podlewanie grzadki {numerGrzadki}")
                self.go_to(numerRobota,5.5,2,f"Zakończyłem podlewanie grzadki {numerGrzadki}")
            elif numerGrzadki == 3:
                self.go_to(numerRobota,8,9,f"Rozpoczynam podlewanie grzadki {numerGrzadki}")
                self.go_to(numerRobota,8,2,f"Zakończyłem podlewanie grzadki {numerGrzadki}")
        else:
            if numerGrzadki == 1:
                self.go_to(numerRobota,3,2,f"Rozpoczynam podlewanie grzadki {numerGrzadki}")
                self.go_to(numerRobota,3,9,f"Zakończyłem podlewanie grzadki {numerGrzadki}")
            elif numerGrzadki == 2:
                self.go_to(numerRobota,5.5,2,f"Rozpoczynam podlewanie grzadki {numerGrzadki}")
                self.go_to(numerRobota,5.5,9,f"Zakończyłem podlewanie grzadki {numerGrzadki}")
            elif numerGrzadki == 3:
                self.go_to(numerRobota,8,2,f"Rozpoczynam podlewanie grzadki {numerGrzadki}")
                self.go_to(numerRobota,8,9,f"Zakończyłem podlewanie grzadki {numerGrzadki}")
        
    # odbieranie poleceń
    def callback_req(self,req):
        arguments = req.data.split(',')
        numerRobota = int(arguments[0].strip()[1])
        
        if numerRobota < 1:
            return
        
        if arguments[1].strip() == "idz":
            x = float(arguments[2])
            y = float(arguments[3])
            komunikat = f"Dojechalem do punktu ({x},{y})"
            self.go_to(numerRobota,x,y,komunikat)
        elif arguments[1].strip() == "naladuj":
            self.charge(numerRobota)
        elif arguments[1].strip() == "podlej":
            nrGrzadki = int(arguments[2])
            self.podlej(numerRobota,nrGrzadki)
        elif arguments[1].strip() == "spirala":
            self.robots[numerRobota - 1].req_pub.publish("spirala")
        '''           
        if arguments[0].strip() == "R1":
            if arguments[1].strip() == "idz":
                x = float(arguments[2])
                y = float(arguments[3])
                komunikat = f"Dojechalem do punktu ({x},{y})"
                self.go_to(1,x,y,komunikat)
            elif arguments[1].strip() == "naladuj":
                self.charge(1)
            elif arguments[1].strip() == "podlej":
                nrGrzadki = int(arguments[2])
                self.podlej(1,nrGrzadki)
            elif arguments[1].strip() == "spirala":
                self.robots[0].req_pub.publish("spirala")
                
        elif arguments[0].strip() == "R2":
            if arguments[1].strip() == "idz":
                x = float(arguments[2])
                y = float(arguments[3])
                komunikat = f"Dojechalem do punktu ({x},{y})"
                self.go_to(2,x,y,komunikat)
            elif arguments[1].strip() == "naladuj":
                self.charge(2)
            elif arguments[1].strip() == "podlej":
                nrGrzadki = int(arguments[2])
                self.podlej(2,nrGrzadki)
            elif arguments[1].strip() == "spirala":
                self.robots[1].req_pub.publish("spirala")
                
        elif arguments[0].strip() == "R3":
            if arguments[1].strip() == "idz":
                x = float(arguments[2])
                y = float(arguments[3])
                komunikat = f"Dojechalem do punktu ({x},{y})"
                self.go_to(3,x,y,komunikat)
            elif arguments[1].strip() == "naladuj":
                self.charge(3)
            elif arguments[1].strip() == "podlej":
                nrGrzadki = int(arguments[2])
                self.podlej(2,nrGrzadki)
            elif arguments[1].strip() == "spirala":
                self.robots[2].req_pub.publish("spirala")
        '''
    def callback_info(self,info):
        self.user_info_pub.publish(info)
        if "Niski poziom baterii!" in info.data:
            numerRobota = int(info.data.split(':')[0][1])
            pos = self.robots[numerRobota - 1].position
            if (not (pos.x >= 9.5 and pos.y >= 9.5)) and (not (pos.x <= 1.5 and pos.y <= 1.5)):
                self.charge(numerRobota)
            
    def listRobots(self):
        for robot in self.robots:
            print(robot.name)
            
    def unregister(self):
        for robot in self.robots:
            robot.unregister()
        self.robots.clear()
        self.req_sub.unregister()
        self.robot_info_sub.unregister()


In [9]:
system = RobotSystem(["R1","R2","R3"])
system.listRobots()

R1
R2
R3


[WARN] [1625757860.951139]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


In [5]:
system.unregister()